In [1]:
from typing import TYPE_CHECKING # 用于静态类型检查阶段的条件导入，不影响运行时
# 延迟模块加载工具，用于提升启动速度或避免不必要的依赖导入
from transformers.utils import _LazyModule
# 用于定义延迟导入的模块结构
from transformers.utils.import_utils import define_import_structure

In [8]:
TYPE_CHECKING

False

In [ ]:
# 一般什么时候用 mypy 这类静态类型检查工具？
# 主要用于中大型 Python 项目，尤其在以下情况：
# 多人协作开发时：明确每个函数参数/返回值类型，防止类型误用；
# 代码复杂、结构庞大时：防止传错参数类型、误用变量等；
# 追求代码健壮性/可靠性时：比如后端系统、框架、工具链；
# 想写“更像 Java/C++”风格的 Python 时：用类型来约束和自解释；
# 需要自动生成文档、IDE 更好补全体验时。
# mypy/类型检查工具都干什么事？
# 它们不会运行程序本身，只分析代码中是否有类型上的错误。例如：
# def add(x: int, y: int) -> int:
#     return x + y

# add(1, 2)        # OK
# add("1", "2")    # mypy 报错：str 类型传给了 int 类型参数
# 这些错误在运行时不会报错（因为 Python 是动态语言），但 mypy 能提前发现。
# 静态类型检查工具像是给动态语言加了一层“类型防护网”，能提前发现很多运行时才会暴露
# 的问题，尤其适合大型项目和多人协作场景。

In [ ]:
# 在静态类型检查时（如使用 mypy），上述代码会被当作有效导入：
# 实际只会导入模块中 __all__ 列表中列出的符号（类、函数、变量等）；
# 如果模块未定义 __all__，则按规则导入所有不以下划线 _ 开头的内容。

In [ ]:
# 如果在类型检查（如 mypy）阶段，则导入全部模块内容供类型系统使用，但这些导入在运行时不执行
# TYPE_CHECKING 是 typing 模块中的一个特殊常量，用于区分**“类型检查时”与“运行时”**的行为。
# 它在运行时总是 False；
# 但在静态类型检查工具（如 mypy, pyright, pyre, pylance）分析代码时，会被认为是 True。
if TYPE_CHECKING:
    from transformers.models.albert.configuration_albert import *
    from transformers.models.albert.modeling_albert import *
    from transformers.models.albert.modeling_flax_albert import *
    from transformers.models.albert.modeling_tf_albert import *
    from transformers.models.albert.tokenization_albert import *
    from transformers.models.albert.tokenization_albert_fast import *
else:
    import sys
    # 使用 LazyModule 在运行时延迟导入模块，仅在实际访问时才真正加载，提高性能并减少依赖冲突
    _file = globals()["__file__"]
    sys.modules[__name__] = _LazyModule(__name__, 
                                        _file, 
                                        define_import_structure(_file),
                                        module_spec=__spec__)

In [13]:
from collections import OrderedDict # 有序字典，保持插入顺序，用于导出 ONNX 时指定结构顺序
from typing import Mapping  # 类型提示：通用的只读映射接口（类似只读 dict）
# HuggingFace 配置基类，保存模型结构/超参数
from transformers.configuration_utils import PretrainedConfig
# HuggingFace ONNX 导出配置基类，用于自定义导出模型的输入输出结构
from transformers.onnx import OnnxConfig

In [14]:
class AlbertConfig(PretrainedConfig):
    r"""
    Examples:

    ```python
    >>> from transformers import AlbertConfig, AlbertModel

    >>> # Initializing an ALBERT-xxlarge style configuration
    >>> albert_xxlarge_configuration = AlbertConfig()

    >>> # Initializing an ALBERT-base style configuration
    >>> albert_base_configuration = AlbertConfig(
    ...     hidden_size=768,
    ...     num_attention_heads=12,
    ...     intermediate_size=3072,
    ... )

    >>> # Initializing a model (with random weights) from the ALBERT-base style configuration
    >>> model = AlbertModel(albert_xxlarge_configuration)

    >>> # Accessing the model configuration
    >>> configuration = model.config
    ```"""
    
    model_type = "albert" # 模型类型

    def __init__(
        self,
        vocab_size=30000,  
        embedding_size=128,
        hidden_size=4096, 
        num_hidden_layers=12, 
        num_hidden_groups=1,
        num_attention_heads=64,
        intermediate_size=16384,
        inner_group_num=1,
        hidden_act="gelu_new",
        hidden_dropout_prob=0,
        attention_probs_dropout_prob=0,
        max_position_embeddings=512,
        type_vocab_size=2,
        initializer_range=0.02,
        layer_norm_eps=1e-12,
        classifier_dropout_prob=0.1,
        position_embedding_type="absolute",
        pad_token_id=0,
        bos_token_id=2,
        eos_token_id=3,
        **kwargs,
    ):
        # 调用父类的init方法
        super().__init__(
            pad_token_id=pad_token_id, bos_token_id=bos_token_id, 
            eos_token_id=eos_token_id, **kwargs)

        self.vocab_size = vocab_size # 词表大小
        self.embedding_size = embedding_size # 词嵌入维度,每个token的表示的维度
        self.hidden_size = hidden_size # 隐藏层维度,每个token的特征维度
        self.num_hidden_layers = num_hidden_layers # 层数
        self.num_hidden_groups = num_hidden_groups # 隐藏层分组数，同组共享参数
        self.num_attention_heads = num_attention_heads  # 每层注意力头的数量
        self.inner_group_num = inner_group_num # 每组内部 attention + FFN 重复次数
        self.hidden_act = hidden_act  # 激活函数，支持字符串或函数（如 gelu_new）
        self.intermediate_size = intermediate_size # 前馈层维度（通常比 hidden_size 大）
        self.hidden_dropout_prob = hidden_dropout_prob # 全连接层的 dropout 概率
        self.attention_probs_dropout_prob = attention_probs_dropout_prob  # 注意力权重的 dropout 概率
        self.max_position_embeddings = max_position_embeddings  # 支持的最大序列长度
        self.type_vocab_size = type_vocab_size # token_type_ids 的词表大小（如句子对任务中）
        self.initializer_range = initializer_range  # 权重初始化的标准差
        self.layer_norm_eps = layer_norm_eps  # LayerNorm 中避免除零的小常数 epsilon
        self.classifier_dropout_prob = classifier_dropout_prob # 分类器前的 dropout 概率
        # 位置编码类型，支持 absolute / relative_key / relative_key_query
        self.position_embedding_type = position_embedding_type

In [15]:
albert_xxlarge_configuration = AlbertConfig() # 默认

In [16]:
albert_base_configuration = AlbertConfig( # base model
     hidden_size=768,
     num_attention_heads=12,
      intermediate_size=3072,
     )

In [ ]:
# model = AlbertModel(albert_xxlarge_configuration)

In [ ]:
class AlbertOnnxConfig(OnnxConfig):
    # 继承自 OnnxConfig，用于配置 ALBERT 模型的 ONNX 导出设置
    # 定义模型的输入信息，指定每个输入的动态轴。
    # 如果任务是多选问题（multiple-choice），则会添加 'choice' 维度作为动态轴。
    # 否则，只会有 'batch' 和 'sequence' 两个动态轴。
    @property 
    def inputs(self) -> Mapping[str, Mapping[int, str]]:
        if self.task == "multiple-choice": # 多选
            # 0:批次维度,1:多选维度,2:序列维度
            dynamic_axis = {0: "batch", 1: "choice", 2: "sequence"} # 动态轴
        else: # 其他任务只有批次和序列维度
            dynamic_axis = {0: "batch", 1: "sequence"}
        return OrderedDict(  # 返回输入字典，映射每个输入到对应的动态轴
            [
                ("input_ids", dynamic_axis),  # 输入 token id
                ("attention_mask", dynamic_axis),  # 注意力掩码
                ("token_type_ids", dynamic_axis), # token 类型 ID（用于区分句子对）
            ]
        )

# 定义在此模块中暴露的符号
__all__ = ["AlbertConfig", "AlbertOnnxConfig"]
# 只允许从该模块导入 AlbertConfig 和 AlbertOnnxConfig

In [ ]:
# inputs 属性：
# inputs 定义了模型的输入（如 input_ids, attention_mask, token_type_ids）以及它们的动态轴。
# 在多选任务中，除了 batch 和 sequence，还会有一个 choice 维度，表示每个选项的不同。